### Jupyter Notebook "2-TF_IDF_Cosine_&_Word2vec_WMD":  
* 3-Imports.
     * 3.1-Import librerias necesarias.
     * 3.2-Import de DFs del Jupyter Notebook anterior.
* 4-División DFs para usarlos en los distintos entrenamientos.
* 5-Realizando Comparaciones y obteniendo Similitudes.
     * 5.1- TF-IDF & Cosine Similarity.
     * 5.2- Word Embedding (Word2vec) & WMD.
* 6-Export del DF para usarlo en el siguiente Jupyter Notebook.

### 3-Imports.

### 3.1-Import librerias necesarias.

In [183]:
import pandas as pd
import numpy as np

#Calculo de TFIDF y Cosine:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import paired_cosine_distances as pcd

#Word2vec: 
import gensim
from gensim.models import Word2Vec

#Para medir tiempos de ejecución:
from datetime import datetime

### 3.2-Import de DFs del Jupyter Notebook anterior.

In [184]:
df_Candidatos_loaded = pd.read_pickle('DF_624_CVs')
df_Candidatos_loaded.shape

(624, 3)

In [185]:
df_Candidatos_loaded.head()

,Candidate_Name,clean_Content_CV,tokens_Content_CV
0,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,..."
1,DataScientist_Rahul_Malik,nlp data_scientist brooklyn ny nlp data_scient...,"[nlp, data_scientist, brooklyn, ny, nlp, data_..."
2,HCM_Federico_Calonge,hcm technical consultant working oracle tool p...,"[hcm, technical, consultant, working, oracle, ..."
3,HCM_Robert_Smith,sap_hcm consultant com qwikresume marshville r...,"[sap_hcm, consultant, com, qwikresume, marshvi..."
4,MLEngineer_Bradly_Johnston,kasey vista detroit senior software_engineer m...,"[kasey, vista, detroit, senior, software_engin..."


In [186]:
df_Jobs_loaded = pd.read_pickle('DF_20593_Job_Desc') # load
df_Jobs_loaded.shape

(20593, 3)

In [187]:
df_Jobs_loaded.head()

,Job_Title,clean_Job_Description,tokens_Job_Description
0,Data Scientist,master_degree stem field including_limited com...,"[master_degree, stem, field, including_limited..."
1,Data Scientist 2,reporting director data analytics senior data_...,"[reporting, director, data, analytics, senior,..."
2,HCM Consultant,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp..."
3,HCM Consultant 2,peoplesoft oracle_eb implementation support hc...,"[peoplesoft, oracle_eb, implementation, suppor..."
4,Machine Learning Engineer,leveraging_latest machine deep_learning techni...,"[leveraging_latest, machine, deep_learning, te..."


### 4-División DFs para usarlos en los distintos entrenamientos.

Los DFs enteros (**df_Candidatos_loaded** - 624 CVs) y (**df_Jobs_loaded** - 20593 Puestos de IT) serán utilizados para entrenar nuestro Vocabulario para el cálculo de WMD. 

Luegos definiremos 2 nuevos DFs para obtener los cálculos de WMD y Cosine Sim.; y para luego entrenar K-means y KNN:   
  * **df_Candidatos_modif** - 301 CVs.  
  * **df_Jobs_modif** - 201 Jobs Descriptions.   

Al realizar los cálculos compararemos cada CV con cada Job Description, obteniendo un dataframe total **df_compar_cand_jobs** de 60501 filas con sus respectivo valores de WMD y Cosine Sim.

Por último eliminaremos 1 CV y 1 Job Description (un total de 501 filas) de **df_compar_cand_jobs**, estos los utilizaremos para testear la predicción por KNN. 

De esta manera nos quedará el **df_compar_cand_jobs** con 60000 filas / puntos para entrenar K-means y KNN.

#### 4.1-Creación de df_Candidatos_modif

In [188]:
df_Candidatos_modif = df_Candidatos_loaded.truncate(before=0, after=100)
df_Candidatos_modif

,Candidate_Name,clean_Content_CV,tokens_Content_CV
0,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,..."
1,DataScientist_Rahul_Malik,nlp data_scientist brooklyn ny nlp data_scient...,"[nlp, data_scientist, brooklyn, ny, nlp, data_..."
2,HCM_Federico_Calonge,hcm technical consultant working oracle tool p...,"[hcm, technical, consultant, working, oracle, ..."
3,HCM_Robert_Smith,sap_hcm consultant com qwikresume marshville r...,"[sap_hcm, consultant, com, qwikresume, marshvi..."
4,MLEngineer_Bradly_Johnston,kasey vista detroit senior software_engineer m...,"[kasey, vista, detroit, senior, software_engin..."
...,...,...,...
96,Naveen Arora,naveen arora current location_chicago il visa ...,"[naveen, arora, current, location_chicago, il,..."
97,Naveen Kumar Bandela,naveen kumar bandela java full_stack developer...,"[naveen, kumar, bandela, java, full_stack, dev..."
98,Naveen.S_Resume,naveen looking contribute design analysis deve...,"[naveen, looking, contribute, design, analysis..."
99,Navneeth Resume,navneet gupta project program_manager pmp pmp_...,"[navneet, gupta, project, program_manager, pmp..."


#### 4.2-Creación de df_Jobs_modif

In [189]:
df_Jobs_modif = df_Jobs_loaded.truncate(before=0, after=30)
df_Jobs_modif

,Job_Title,clean_Job_Description,tokens_Job_Description
0,Data Scientist,master_degree stem field including_limited com...,"[master_degree, stem, field, including_limited..."
1,Data Scientist 2,reporting director data analytics senior data_...,"[reporting, director, data, analytics, senior,..."
2,HCM Consultant,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp..."
3,HCM Consultant 2,peoplesoft oracle_eb implementation support hc...,"[peoplesoft, oracle_eb, implementation, suppor..."
4,Machine Learning Engineer,leveraging_latest machine deep_learning techni...,"[leveraging_latest, machine, deep_learning, te..."
5,Machine Learning Engineer 2,collaborate multidisciplinary_team gain_insigh...,"[collaborate, multidisciplinary_team, gain_ins..."
6,Security Specialist,fast_paced environment combine technical secur...,"[fast_paced, environment, combine, technical, ..."
7,Security Specialist 2,handling incoming_request assistance business ...,"[handling, incoming_request, assistance, busin..."
8,Web Developer Full Stack,graduate_degree information technology similar...,"[graduate_degree, information, technology, sim..."
9,Web Developer Full Stack 2,enter existing codebases extend functionality ...,"[enter, existing, codebases, extend, functiona..."


###  5-Realizando Comparaciones y obteniendo Similitudes.

###### Creamos un nuevo DF concatenando los 2 dataframe de manera que se comparen todos los CVs con todos los Jobs:

In [190]:
df_compar_cand_jobs = pd.merge(df_Candidatos_modif.assign(A=1), df_Jobs_modif.assign(A=1), on='A').drop('A', 1)
df_compar_cand_jobs

,Candidate_Name,clean_Content_CV,tokens_Content_CV,Job_Title,clean_Job_Description,tokens_Job_Description
0,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Data Scientist,master_degree stem field including_limited com...,"[master_degree, stem, field, including_limited..."
1,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Data Scientist 2,reporting director data analytics senior data_...,"[reporting, director, data, analytics, senior,..."
2,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",HCM Consultant,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp..."
3,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",HCM Consultant 2,peoplesoft oracle_eb implementation support hc...,"[peoplesoft, oracle_eb, implementation, suppor..."
4,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Machine Learning Engineer,leveraging_latest machine deep_learning techni...,"[leveraging_latest, machine, deep_learning, te..."
...,...,...,...,...,...,...
3126,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...",Test Lead / Test Manager,location_atlanta gajob title test lead manager...,"[location_atlanta, gajob, title, test, lead, m..."
3127,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...",Principal Application Analyst-Supply Chain Lawson,vanderhouwen_job like find please join talent ...,"[vanderhouwen_job, like, find, please, join, t..."
3128,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...",Messaging Administrator,genesis actively_seeking messaging administrat...,"[genesis, actively_seeking, messaging, adminis..."
3129,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...","Java Architect - Denver, CO - Fulltime",client healthcare software industry seeking ja...,"[client, healthcare, software, industry, seeki..."


### 5.1- TF-IDF & Cosine Similarity.

#### Definimos los vectores que utilizaremos para entrenar y evaluar TF-IDF / Cosine Sim:

In [191]:
#Para entrenar tf-idf usamos todo el vocabulario:
array_unicode_CV_train   = df_Candidatos_loaded['clean_Content_CV'].astype('U').values
array_unicode_Jobs_train = df_Jobs_loaded['clean_Job_Description'].astype('U').values

print(len(array_unicode_CV_train))
print(array_unicode_CV_train[0])       #Primer valor de la lista.
print(len(array_unicode_Jobs_train))
print(array_unicode_Jobs_train[20592]) #Último valor de la lista.

#Juntamos los vectores:
array_CV_and_Jobs_train = np.concatenate((array_unicode_CV_train, array_unicode_Jobs_train)) #np = numpy.
print(len(array_CV_and_Jobs_train)) 

print('###################################################################################')

#Y para transform usamos únicamente los que queremos evaluar:
array_unicode_CV_transf   = df_compar_cand_jobs['clean_Content_CV'].astype('U').values
array_unicode_Jobs_transf = df_compar_cand_jobs['clean_Job_Description'].astype('U').values
print(len(array_unicode_CV_transf))
print(len(array_unicode_Jobs_transf))

624
data_scientist brooklyn ny data_scientist grubhub current new_york ny implemented various time_series forecasting technique predict surge customer order lower average customer wait time minute github deployed recommendation engine production conditionally recommend menu item based past order history increase average order size designed model portland pilot program increase incentive driver peak ordering hour resulting_increase driver availability peak ordering time statistic rutgers university data_scientist new brunswick_nj adobe new_york ny worked product marketing team identify customer interaction free trial maximize likelihood conversion resulting conversion_rate increase built customer attrition random forest model improved monthly python numpy_panda scikit_learn retention basis point customer likely attrit kera flask servicing relevant product feature sql mysql postgres worked_closely product team build production git recommendation engine python improved average length user

#### Entrenamos TF-IDF y lo aplicamos junto a Cosine Sim

In [192]:
#https://www.py4u.net/discuss/188191
    
#Ahora calcularemos cosine similarity usando vectores tf-idf entre 'search queries' y 'matched documents'.
#Mi 'search query' es la columna de clean_content_CV y el 'matched documents' son los clean_Job_Description.
#Al hacer el FIT entrenamos nuestro modelo con los cvs y job descriptions. Este modelo luego lo usaremos 
#al hacer el TRANSFORM donde le pasamos la query para que se fije los TF-IDF que tiene ya entrenados del modelo. 

#VEr imports:
    #TfidfVectorizer
    #pcd  (cosine)

# Inicializamos una instancia de tf-idf Vectorizer:
tfidf_vectorizer = TfidfVectorizer()

# Generamos los vectores tf-idf para el Corpus:
tfidf_vectorizer.fit(array_CV_and_Jobs_train)             #array_CV_and_Jobs_train obtenido en la celda anterior.

A = tfidf_vectorizer.transform(array_unicode_CV_transf)   #array_unicode_CV_transf obtenido en la celda anterior. 
B = tfidf_vectorizer.transform(array_unicode_Jobs_transf) #array_unicode_Jobs_transf obtenido en la celda anterior. 

#Calculamos cosine similarity:
    #Parametros de pcd:
        #Xarray-like of shape (n_samples, n_features)
        #Yarray-like of shape (n_samples, n_features)
    #Retorna:
        #distancesndarray of shape (n_samples,)

cosine = 1 - pcd(A, B)
df_compar_cand_jobs['tfidf_cosine'] = cosine

df_compar_cand_jobs

,Candidate_Name,clean_Content_CV,tokens_Content_CV,Job_Title,clean_Job_Description,tokens_Job_Description,tfidf_cosine
0,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Data Scientist,master_degree stem field including_limited com...,"[master_degree, stem, field, including_limited...",0.042817
1,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Data Scientist 2,reporting director data analytics senior data_...,"[reporting, director, data, analytics, senior,...",0.111344
2,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",HCM Consultant,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...",0.018021
3,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",HCM Consultant 2,peoplesoft oracle_eb implementation support hc...,"[peoplesoft, oracle_eb, implementation, suppor...",0.013100
4,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Machine Learning Engineer,leveraging_latest machine deep_learning techni...,"[leveraging_latest, machine, deep_learning, te...",0.050443
...,...,...,...,...,...,...,...
3126,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...",Test Lead / Test Manager,location_atlanta gajob title test lead manager...,"[location_atlanta, gajob, title, test, lead, m...",0.025742
3127,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...",Principal Application Analyst-Supply Chain Lawson,vanderhouwen_job like find please join talent ...,"[vanderhouwen_job, like, find, please, join, t...",0.105683
3128,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...",Messaging Administrator,genesis actively_seeking messaging administrat...,"[genesis, actively_seeking, messaging, adminis...",0.063452
3129,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...","Java Architect - Denver, CO - Fulltime",client healthcare software industry seeking ja...,"[client, healthcare, software, industry, seeki...",0.080821


In [193]:
#Vemos por ejemplo los valores para el Candidato 'MLEngineer_Bradly_Johnston':

is_Bradly_Johnston =  df_compar_cand_jobs['Candidate_Name']=='MLEngineer_Bradly_Johnston'
new_DF = df_compar_cand_jobs[is_Bradly_Johnston]
print(new_DF.shape)
new_DF.head(10)                                                               

(31, 7)


,Candidate_Name,clean_Content_CV,tokens_Content_CV,Job_Title,clean_Job_Description,tokens_Job_Description,tfidf_cosine
124,MLEngineer_Bradly_Johnston,kasey vista detroit senior software_engineer m...,"[kasey, vista, detroit, senior, software_engin...",Data Scientist,master_degree stem field including_limited com...,"[master_degree, stem, field, including_limited...",0.159509
125,MLEngineer_Bradly_Johnston,kasey vista detroit senior software_engineer m...,"[kasey, vista, detroit, senior, software_engin...",Data Scientist 2,reporting director data analytics senior data_...,"[reporting, director, data, analytics, senior,...",0.313058
126,MLEngineer_Bradly_Johnston,kasey vista detroit senior software_engineer m...,"[kasey, vista, detroit, senior, software_engin...",HCM Consultant,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...",0.041684
127,MLEngineer_Bradly_Johnston,kasey vista detroit senior software_engineer m...,"[kasey, vista, detroit, senior, software_engin...",HCM Consultant 2,peoplesoft oracle_eb implementation support hc...,"[peoplesoft, oracle_eb, implementation, suppor...",0.039275
128,MLEngineer_Bradly_Johnston,kasey vista detroit senior software_engineer m...,"[kasey, vista, detroit, senior, software_engin...",Machine Learning Engineer,leveraging_latest machine deep_learning techni...,"[leveraging_latest, machine, deep_learning, te...",0.246166
129,MLEngineer_Bradly_Johnston,kasey vista detroit senior software_engineer m...,"[kasey, vista, detroit, senior, software_engin...",Machine Learning Engineer 2,collaborate multidisciplinary_team gain_insigh...,"[collaborate, multidisciplinary_team, gain_ins...",0.325581
130,MLEngineer_Bradly_Johnston,kasey vista detroit senior software_engineer m...,"[kasey, vista, detroit, senior, software_engin...",Security Specialist,fast_paced environment combine technical secur...,"[fast_paced, environment, combine, technical, ...",0.038583
131,MLEngineer_Bradly_Johnston,kasey vista detroit senior software_engineer m...,"[kasey, vista, detroit, senior, software_engin...",Security Specialist 2,handling incoming_request assistance business ...,"[handling, incoming_request, assistance, busin...",0.025728
132,MLEngineer_Bradly_Johnston,kasey vista detroit senior software_engineer m...,"[kasey, vista, detroit, senior, software_engin...",Web Developer Full Stack,graduate_degree information technology similar...,"[graduate_degree, information, technology, sim...",0.035656
133,MLEngineer_Bradly_Johnston,kasey vista detroit senior software_engineer m...,"[kasey, vista, detroit, senior, software_engin...",Web Developer Full Stack 2,enter existing codebases extend functionality ...,"[enter, existing, codebases, extend, functiona...",0.053284


### 5.2- Word Embedding (Word2vec) & WMD.

#### Usamos un Word2vec entrenado con nuestro vocabulario.


#### Unimos las columnas 'tokens_Job_Description' de 'df_Jobs_loaded' y 'tokens_Content_CV' de 'df_Candidatos_loaded' generando una lista de listas, la cual será nuestro vocabulario que introduciremos a Word EMbedding como entrenamiento:

In [194]:
list_vocab_Jobs = df_Jobs_loaded['tokens_Job_Description']

In [195]:
list_vocab_Jobs

0        [master_degree, stem, field, including_limited...
1        [reporting, director, data, analytics, senior,...
2        [oracle, cloud, hcm, absence, consultant, resp...
3        [peoplesoft, oracle_eb, implementation, suppor...
4        [leveraging_latest, machine, deep_learning, te...
                               ...                        
20588    [company, description, searching, talented, cr...
20589    [location_san, francisco, caterm, full_time, p...
20590    [take_pride, knowing, thousand, life, positive...
20591    [company, description, offer, youas, world_lea...
20592    [c_c, programmingdevelopment, win, programming...
Name: tokens_Job_Description, Length: 20593, dtype: object

In [196]:
list_vocab_Candid = df_Candidatos_loaded['tokens_Content_CV']
#list_vocab_Candid.tolist()

In [197]:
list_vocab_Candid

0      [data_scientist, brooklyn, ny, data_scientist,...
1      [nlp, data_scientist, brooklyn, ny, nlp, data_...
2      [hcm, technical, consultant, working, oracle, ...
3      [sap_hcm, consultant, com, qwikresume, marshvi...
4      [kasey, vista, detroit, senior, software_engin...
                             ...                        
619    [technical, c_asp, net, mvc, html_cs, javascri...
620    [detail, detail, pune_maharashtra, university,...
621    [technology, asp_net, mvc, unit_testing, entit...
622    [technical, category, language, c_c, oop, dot_...
623    [technical, programming_c, net, web, technolog...
Name: tokens_Content_CV, Length: 624, dtype: object

In [198]:
all_vocab = list_vocab_Candid.append(list_vocab_Jobs) #Unimos las listas.

In [199]:
all_vocab

0        [data_scientist, brooklyn, ny, data_scientist,...
1        [nlp, data_scientist, brooklyn, ny, nlp, data_...
2        [hcm, technical, consultant, working, oracle, ...
3        [sap_hcm, consultant, com, qwikresume, marshvi...
4        [kasey, vista, detroit, senior, software_engin...
                               ...                        
20588    [company, description, searching, talented, cr...
20589    [location_san, francisco, caterm, full_time, p...
20590    [take_pride, knowing, thousand, life, positive...
20591    [company, description, offer, youas, world_lea...
20592    [c_c, programmingdevelopment, win, programming...
Length: 21217, dtype: object

In [200]:
lista_all_vocab = all_vocab.tolist()
lista_all_vocab[0]

['data_scientist',
 'brooklyn',
 'ny',
 'data_scientist',
 'grubhub',
 'current',
 'new_york',
 'ny',
 'implemented',
 'various',
 'time_series',
 'forecasting',
 'technique',
 'predict',
 'surge',
 'customer',
 'order',
 'lower',
 'average',
 'customer',
 'wait',
 'time',
 'minute',
 'github',
 'deployed',
 'recommendation',
 'engine',
 'production',
 'conditionally',
 'recommend',
 'menu',
 'item',
 'based',
 'past',
 'order',
 'history',
 'increase',
 'average',
 'order',
 'size',
 'designed',
 'model',
 'portland',
 'pilot',
 'program',
 'increase',
 'incentive',
 'driver',
 'peak',
 'ordering',
 'hour',
 'resulting_increase',
 'driver',
 'availability',
 'peak',
 'ordering',
 'time',
 'statistic',
 'rutgers',
 'university',
 'data_scientist',
 'new',
 'brunswick_nj',
 'adobe',
 'new_york',
 'ny',
 'worked',
 'product',
 'marketing',
 'team',
 'identify',
 'customer',
 'interaction',
 'free',
 'trial',
 'maximize',
 'likelihood',
 'conversion',
 'resulting',
 'conversion_rate',
 'i

In [201]:
len(lista_all_vocab)

21217

#### Modelo word2vec:

In [202]:
#Ver imports:
    #gensim
    #Word2Vec

#La librería Gensim nos provee una simple API al algoritmo de Google 'word2vec' el cual es usado para crear word embeddings.
#Word2vec son modelos de redes neuronales superficiales de dos capas que están entrenadas para
#reconstruir contextos lingüísticos de palabras. 

#Podemos traer word embeddings pre-entrenados con noticias de Google por ejemplo, donde se entrenarán con 3 millones de palabras.
#Pero, al comprobar que este vocabulario no tiene palabras como "data_science" o "machine_learning" 
#lo que hicimos fue utilizar nuestro vocabulario para entrenar y crear nuestros propios embeddings.  

#La entrada de la red neuronal es una palabra representada como un vector “one-hot”, es decir, un vector con tantas posiciones 
#como tamaño tenga el vocabulario. 
#Por ejemplo, si queremos representar la palabra “sol” de un vocabulario de 4 palabras (el sol está cayendo), usaremos un vector
#de de esta dimensión (5) con cero en todas las posiciones menos la correspondiente a la palabra “sol” que tendrá 
#un uno --> [0,1,0,0]
#Y la  salida de la red neuronal será otro vector “one-hot” llamado “Word embedding” de las mismas 4 posiciones que representará
#las probabilidades de cada una de las palabras sean vecinas de la palabra representada en la entrada. [0.12, 1, 0.4, 0.1]
#--> Es 1 en la 2da posición ya que es la misma posición de la palabra. 

#Word embedding es una de las representaciones de un documento de vocabulario más popular. Es capaz de caputar los
#contextos de una palabra en el documento, similitud semantica y sintáctica, relaciones con otras palabras, etc.

#Mediante la siguiente línea de código entrenamos al modelo Word2Vec mediante nuestras 21217 listas de palabras contenidas en “lista_all_vocab”:
model=gensim.models.Word2Vec(sentences=lista_all_vocab,min_count=2,workers=4,window=4,sg=0)
#En este caso no dividimos en train y test, ya que no vamos a predecir palabras en base a contextos, 
#lo que haremos es utilizar este word embedding para calcular WMD, entonces usamos TODAS las sentencias como entrenamiento. 

#model = Word2Vec(sentences=lista_all_vocab, size=200, window=4, min_count=1, workers=4)
#Parametros funcion Word2Vec:
#min_count : Ignora todas las palabras cuando la frecuencia de cada palabra es menor a 'min_count'. El valor por
             #defecto es 5. Si queremos añadir todas las palabras en nuestro corpus colocaremos un valor de '1'.
#size: la longitud del vector denso para representar cada palabra. #VER.
#window: es el tamaño del "filtro" que se usa para analizar el contexto entre una palbras y las otras.  #VER. 
#workers: las iteraciones de entrenamiento. #VER. 
#sg: es si utiliza el algoritmo de entrenamiento CBOW o skip gram. Por defecto es 0 (CBOW).

model.save("model")           #Guardamos nuestro modelo.
wrds=list(model.wv.vectors)   #El modelo nos da un vocabulario con palabras con las cuales podemos consultar. 
print(len(wrds)) #Me devuelve las palabras/tokens totales y ÚNICAS (sin repetidas) de mi vocabulario.

60713


In [203]:
df_compar_cand_jobs

,Candidate_Name,clean_Content_CV,tokens_Content_CV,Job_Title,clean_Job_Description,tokens_Job_Description,tfidf_cosine
0,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Data Scientist,master_degree stem field including_limited com...,"[master_degree, stem, field, including_limited...",0.042817
1,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Data Scientist 2,reporting director data analytics senior data_...,"[reporting, director, data, analytics, senior,...",0.111344
2,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",HCM Consultant,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...",0.018021
3,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",HCM Consultant 2,peoplesoft oracle_eb implementation support hc...,"[peoplesoft, oracle_eb, implementation, suppor...",0.013100
4,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Machine Learning Engineer,leveraging_latest machine deep_learning techni...,"[leveraging_latest, machine, deep_learning, te...",0.050443
...,...,...,...,...,...,...,...
3126,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...",Test Lead / Test Manager,location_atlanta gajob title test lead manager...,"[location_atlanta, gajob, title, test, lead, m...",0.025742
3127,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...",Principal Application Analyst-Supply Chain Lawson,vanderhouwen_job like find please join talent ...,"[vanderhouwen_job, like, find, please, join, t...",0.105683
3128,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...",Messaging Administrator,genesis actively_seeking messaging administrat...,"[genesis, actively_seeking, messaging, adminis...",0.063452
3129,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...","Java Architect - Denver, CO - Fulltime",client healthcare software industry seeking ja...,"[client, healthcare, software, industry, seeki...",0.080821


In [204]:
#De esta manera debería ser lo que usemos dentro de la función wmd():
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
print(type(sentence_obama))
sentence_obama

<class 'list'>


['obama', 'speaks', 'to', 'the', 'media', 'in', 'illinois']

In [205]:
#Vemos que nuestra columna está en el formato CORRECTO:
primer_job_desc = df_compar_cand_jobs.loc[0,'tokens_Job_Description']
print(type(primer_job_desc))
primer_job_desc

<class 'list'>


['master_degree',
 'stem',
 'field',
 'including_limited',
 'computer_science',
 'statistic_applied',
 'mathematics',
 'operation',
 'research',
 'engineering',
 'economics',
 'social',
 'science',
 'physic',
 'chemistry',
 'providing',
 'advanced',
 'analytics',
 'within',
 'business',
 'setting',
 'data',
 'science',
 'implementation',
 'within',
 'business',
 'setting',
 'working',
 'raw',
 'missing',
 'data',
 'understanding',
 'programming',
 'fundamental',
 'understanding',
 'statistic',
 'excellent_communication',
 'preferred',
 'data_visualization',
 'tool',
 'time',
 'series',
 'working',
 'knowledge',
 'relational_database',
 'standard',
 'sql_query',
 'method',
 'proficiency_least',
 'one',
 'general',
 'programming_language',
 'python',
 'java',
 'c_c',
 'working',
 'big_data',
 'within',
 'hadoop',
 'environment',
 'working',
 'r_sa',
 'statistical',
 'package',
 'advanced',
 'statistical',
 'econometric',
 'data_mining',
 'tool',
 'method',
 'linear',
 'model',
 'linear_r

In [206]:
model.wv.get_vector('machine_learning')

array([-0.99525577, -0.6077188 ,  0.49346033, -0.55763036, -0.9796547 ,
       -0.68836665, -0.71350926, -1.4117522 ,  1.8915818 , -0.6506267 ,
        0.42556623, -0.26373655, -1.1735786 ,  0.51572156,  1.8493272 ,
       -0.7794592 ,  0.2914408 , -0.72422415, -2.4389746 , -0.6153202 ,
       -1.5892342 ,  1.0106893 , -1.0484846 ,  2.2489483 , -0.21412507,
        2.9172063 , -0.96349007, -1.3327242 ,  0.00416835,  1.3986703 ,
        1.4242766 , -0.22011311,  2.223857  , -1.1588553 , -1.4619132 ,
        0.1856661 , -1.7423491 ,  0.8147439 , -2.6550345 , -1.3177652 ,
        0.27040812,  0.5305417 ,  0.9915678 ,  0.22363521,  1.6627038 ,
       -2.082534  ,  0.45118788,  0.05226795, -0.7729668 , -0.7210281 ,
       -1.7114677 , -0.26790208, -0.13104407, -1.8135256 , -0.37597892,
       -0.50595236,  0.21065246,  0.22731315, -1.0537595 ,  0.47375342,
        0.6429516 ,  0.2106793 , -0.45202067,  0.09661713, -0.5713031 ,
        1.3932399 , -1.6853856 ,  3.1746583 , -1.5431623 ,  0.79

In [207]:
model.wv.wmdistance('deep_learning', 'machine_learning')

0.3127839378571345

In [208]:
#Gracias a nuestros words embeddings obtenidos por el entrenamiento en el modelo word2vec,
#ahora podemos realizar funciones como 'most_similar'....testeamos nuestro modelo: 
ModeloPrueba=model.wv.most_similar("machine_learning")     

#La función most_similar busca palabras que esten semánticamente
#cercas (que sean parecidas) a una palabra dada (En este caso 'machine_learning').

In [209]:
print(ModeloPrueba)

[('deep_learning', 0.8419713377952576), ('artificial_intelligence', 0.8369033336639404), ('predictive_analytics', 0.834757387638092), ('computational', 0.8211137652397156), ('algorithm', 0.8190171718597412), ('data_mining', 0.8028570413589478), ('nlp', 0.7969160079956055), ('predictive', 0.7912158966064453), ('predictive_modeling', 0.7865807414054871), ('ml', 0.773136556148529)]


In [210]:
#Función WMD que usaremos:
def WMD(tokens_CV_Candidate,tokens_Job_Desc):
    return (model.wv.wmdistance(tokens_CV_Candidate, tokens_Job_Desc))  #'wmdistance' return the word mover distance between two documents. 

In [211]:
#Como prueba obtenemos el WMD comparando para la 1ra fila (entre las columnas tokens_Content_CV y tokens_Job_Description)
first_job_descr = df_compar_cand_jobs.loc[0,'tokens_Job_Description']
first_content_cv = df_compar_cand_jobs.loc[0,'tokens_Content_CV']

wmd_result = round(WMD(first_content_cv,first_job_descr),3)
print(wmd_result)
 
similarity_wdm = round((1/(1+wmd_result)),3)  
print(similarity_wdm)

0.934
0.517


In [212]:
#Aplicamos WMD para TODO el DF... entre el contenido del CV y la descripcion del puesto; 
#y el resultado lo guardamos en las columna 'WMD'.

#df_compar_cand_jobs['WMD'] = df_compar_cand_jobs.apply(lambda row: round(WMD(row['tokens_Content_CV'],row['tokens_Job_Description']),3), axis=1)
#APlicando lo de  --> similarity = 1 / (1 + distance):
#(#similarity = 1 / (1 + distance) https://groups.google.com/g/gensim/c/-pRZnsOEaPQ)

start_time = datetime.now()

df_compar_cand_jobs['WMD'] = df_compar_cand_jobs.apply(lambda row: round((1/(1+(WMD(row['tokens_Content_CV'],row['tokens_Job_Description'])))),3), axis=1)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 7:00:27.420719


In [214]:
df_compar_cand_jobs

,Candidate_Name,clean_Content_CV,tokens_Content_CV,Job_Title,clean_Job_Description,tokens_Job_Description,tfidf_cosine,WMD
0,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Data Scientist,master_degree stem field including_limited com...,"[master_degree, stem, field, including_limited...",0.042817,0.517
1,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Data Scientist 2,reporting director data analytics senior data_...,"[reporting, director, data, analytics, senior,...",0.111344,0.525
2,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",HCM Consultant,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...",0.018021,0.501
3,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",HCM Consultant 2,peoplesoft oracle_eb implementation support hc...,"[peoplesoft, oracle_eb, implementation, suppor...",0.013100,0.500
4,DataScientist_Karla_Lewis,data_scientist brooklyn ny data_scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Machine Learning Engineer,leveraging_latest machine deep_learning techni...,"[leveraging_latest, machine, deep_learning, te...",0.050443,0.518
...,...,...,...,...,...,...,...,...
3126,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...",Test Lead / Test Manager,location_atlanta gajob title test lead manager...,"[location_atlanta, gajob, title, test, lead, m...",0.025742,0.485
3127,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...",Principal Application Analyst-Supply Chain Lawson,vanderhouwen_job like find please join talent ...,"[vanderhouwen_job, like, find, please, join, t...",0.105683,0.516
3128,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...",Messaging Administrator,genesis actively_seeking messaging administrat...,"[genesis, actively_seeking, messaging, adminis...",0.063452,0.512
3129,Neha Mugghala,sourya senior java_ee developer working variou...,"[sourya, senior, java_ee, developer, working, ...","Java Architect - Denver, CO - Fulltime",client healthcare software industry seeking ja...,"[client, healthcare, software, industry, seeki...",0.080821,0.507


### Vemos como ejemplo a Federico Calonge:

In [70]:
is_Federico_Calonge =  df_compar_cand_jobs['Candidate_Name']=='DataScientist_Rahul_Malik'
new_DF = df_compar_cand_jobs[is_Federico_Calonge]
print(new_DF.shape)
new_DF

(201, 7)


,Candidate_Name,clean_Content_CV,tokens_Content_CV,Job_Title,clean_Job_Description,tokens_Job_Description,tfidf_cosine
201,DataScientist_Rahul_Malik,nlp data_scientist brooklyn ny nlp data_scient...,"['nlp', 'data_scientist', 'brooklyn', 'ny', 'n...",Data Scientist,master_degree stem field including_limited com...,"['master_degree', 'stem', 'field', 'including_...",0.024535
202,DataScientist_Rahul_Malik,nlp data_scientist brooklyn ny nlp data_scient...,"['nlp', 'data_scientist', 'brooklyn', 'ny', 'n...",Data Scientist 2,reporting director data analytics senior data_...,"['reporting', 'director', 'data', 'analytics',...",0.097801
203,DataScientist_Rahul_Malik,nlp data_scientist brooklyn ny nlp data_scient...,"['nlp', 'data_scientist', 'brooklyn', 'ny', 'n...",HCM Consultant,oracle cloud hcm absence consultant responsibl...,"['oracle', 'cloud', 'hcm', 'absence', 'consult...",0.016022
204,DataScientist_Rahul_Malik,nlp data_scientist brooklyn ny nlp data_scient...,"['nlp', 'data_scientist', 'brooklyn', 'ny', 'n...",HCM Consultant 2,peoplesoft oracle_eb implementation support hc...,"['peoplesoft', 'oracle_eb', 'implementation', ...",0.003291
205,DataScientist_Rahul_Malik,nlp data_scientist brooklyn ny nlp data_scient...,"['nlp', 'data_scientist', 'brooklyn', 'ny', 'n...",Machine Learning Engineer,leveraging_latest machine deep_learning techni...,"['leveraging_latest', 'machine', 'deep_learnin...",0.044236
...,...,...,...,...,...,...,...
397,DataScientist_Rahul_Malik,nlp data_scientist brooklyn ny nlp data_scient...,"['nlp', 'data_scientist', 'brooklyn', 'ny', 'n...",Web developer (AWS and NodeJS),detail ideal_candidate software development pr...,"['detail', 'ideal_candidate', 'software', 'dev...",0.011928
398,DataScientist_Rahul_Malik,nlp data_scientist brooklyn ny nlp data_scient...,"['nlp', 'data_scientist', 'brooklyn', 'ny', 'n...",Counselor- Project/Program (Admin),tad_pgs inc_currently seeking counselor projec...,"['tad_pgs', 'inc_currently', 'seeking', 'couns...",0.023195
399,DataScientist_Rahul_Malik,nlp data_scientist brooklyn ny nlp data_scient...,"['nlp', 'data_scientist', 'brooklyn', 'ny', 'n...",Production Shipper,tad_pgs inc_currently seeking general laborer ...,"['tad_pgs', 'inc_currently', 'seeking', 'gener...",0.020360
400,DataScientist_Rahul_Malik,nlp data_scientist brooklyn ny nlp data_scient...,"['nlp', 'data_scientist', 'brooklyn', 'ny', 'n...",PeopleSoft ELM Developer:,peoplesoft elm developer enterprise learning m...,"['peoplesoft', 'elm', 'developer', 'enterprise...",0.006975


Observamos altos valores de similitud para los puestos 'HCM Consultant' y 'HCM Consultant 2' (algo esperado).

In [245]:
is_Bradly_Johnston =  df_compar_cand_jobs['Candidate_Name']=='MLEngineer_Bradly_Johnston'
new_DF = df_compar_cand_jobs[is_Bradly_Johnston]
print(new_DF.shape)
new_DF

(10, 10)


,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,tfidf_cosine,WMD_Job_Desc
40,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"['master', 'degree', 'stem', 'field', 'includi...",0.196025,0.924
41,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"['reporting', 'director', 'data', 'analytics',...",0.346808,0.910
42,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"['oracle', 'cloud', 'hcm', 'absence', 'consult...",0.059270,0.898
43,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...,peoplesoft oracle eb implementation support hc...,"['peoplesoft', 'oracle', 'eb', 'implementation...",0.058595,0.923
44,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Machine Learning Engineer,Leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"['leveraging', 'latest', 'machine', 'deep_lear...",0.286638,0.924
45,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"['collaborate', 'multidisciplinary', 'team', '...",0.333604,0.941
46,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Security Specialist,Work in a fast-paced environment that combine ...,fast paced environment combine technical secur...,"['fast', 'paced', 'environment', 'combine', 't...",0.064902,0.918
47,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming request assistance business ...,"['handling', 'incoming', 'request', 'assistanc...",0.044034,0.898
48,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Web Developer Full Stack,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"['graduate', 'degree', 'information', 'technol...",0.048537,0.905
49,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Web Developer Full Stack 2,\n· Enter existing website codebases and exten...,enter existing codebases extend functionality ...,"['enter', 'existing', 'codebases', 'extend', '...",0.066777,0.927


### 6-Export del DF para usarlo en el siguiente Jupyter Notebook.

In [213]:
#Exportamos nuestro modelo:
df_compar_cand_jobs.to_pickle('DF_Exportado_Jobs_And_Candidates_own_model')